## Segment a sparse 3D image with a single material component  

The goal of this notebook is to develop a 3D segmentation algorithm that improves segmentation where features are detected.

**Data:** AM parts from Xuan Zhang. 

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import h5py
import sys
import time
import seaborn as sns
import pandas as pd

import cupy as cp
from tomo_encoders import Patches
from tomo_encoders.misc import viewer
from tomo_encoders import DataFile
from tomo_encoders.reconstruction.recon import recon_binning, recon_patches_3d, rec_patch, rec_pts
# from tomo_encoders.misc.voxel_processing import cylindrical_mask, normalize_volume_gpu

In [2]:
r_fac = 1.0
ht = 32
wd = 2176
th = 1500
n_sel =int(ht*wd*wd*r_fac)

In [3]:
data = cp.random.normal(0,1,(th, ht, wd), dtype = cp.float32)
theta = cp.linspace(0, np.pi, th, dtype = cp.float32)
center = wd/2
vol = np.ones((ht,wd,wd))
vox_coords = np.where(vol == 1)
pts = np.asarray(vox_coords).T
pts = cp.asarray(pts, dtype = cp.int32)
pts = pts[cp.random.choice(len(pts), n_sel, replace = False),...].copy()

In [4]:
t000 = time.time()
gpts = pts[:,0]*wd*wd+pts[:,1]*wd+pts[:,2]
ind = cp.argsort(gpts)
pts = pts[ind]
t_sort = (time.time()-t000)*1000.0
print(f'sorting overhead: {t_sort:.2f} ms')

sorting overhead: 217.09 ms


In [5]:
for i in range(5):
    obj1 = rec_pts(data, theta, center, pts)
obj1 = obj1.reshape(ht,wd,wd)

TIME rec_pts: 1373.76 ms
TIME rec_pts: 1378.25 ms
TIME rec_pts: 1380.82 ms
TIME rec_pts: 1382.40 ms
TIME rec_pts: 1387.17 ms


In [6]:
times = []
for i in range(5):
    obj, t_ = rec_patch(data, theta, center, 0, wd, 0, wd, 0, ht, TIMEIT=True)
    print(f"time {t_:.2f} ms")
    times.append(t_)
print(f"time = {np.median(times):.2f} ms")    

time 1313.07 ms
time 1322.16 ms
time 1326.68 ms
time 1324.04 ms
time 1329.88 ms
time = 1324.04 ms


In [7]:
obj.size

151519232

In [8]:
print(f'is algorithm working fine? {~np.any(obj1-obj)}')

is algorithm working fine? True
